# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116584e+02     1.555192e+00
 * time: 0.06557297706604004
     1     1.033659e+01     9.556110e-01
 * time: 1.2989130020141602
     2    -1.200258e+01     1.061363e+00
 * time: 1.3850629329681396
     3    -3.439596e+01     8.183056e-01
 * time: 1.4652128219604492
     4    -4.782606e+01     6.759891e-01
 * time: 1.5530569553375244
     5    -5.694605e+01     2.679392e-01
 * time: 1.6400988101959229
     6    -5.968804e+01     2.424373e-01
 * time: 1.6963229179382324
     7    -6.086131e+01     9.734727e-02
 * time: 1.7515900135040283
     8    -6.140319e+01     5.223618e-02
 * time: 1.8159217834472656
     9    -6.170959e+01     3.380191e-02
 * time: 1.8711168766021729
    10    -6.189568e+01     3.814812e-02
 * time: 1.9273579120635986
    11    -6.200385e+01     2.173575e-02
 * time: 1.9843389987945557
    12    -6.208674e+01     1.676159e-02
 * time: 2.0467779636383057
    13    -6.214347e+01     1.537639e-02
 * time: 2.104712963

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557668
    AtomicNonlocal      14.8522635
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336820

    total               -62.261666462131
